In [80]:
import folium
import pandas as pd
import numpy as np

folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="CartoDB dark_matter")

bike_data = pd.read_csv("/home/tallys/git/master-research/src/datasets/citibike/JC-201610-citibike-tripdata.csv")
bike_data.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,183,2016-10-01 00:00:38,2016-10-01 00:03:41,3272,Jersey & 3rd,40.723332,-74.045953,3203,Hamilton Park,40.727596,-74.044247,24510,Subscriber,1968.0,1
1,379,2016-10-01 00:04:57,2016-10-01 00:11:17,3273,Manila & 1st,40.721651,-74.042884,3272,Jersey & 3rd,40.723332,-74.045953,26152,Subscriber,1987.0,1
2,414,2016-10-01 00:05:18,2016-10-01 00:12:13,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,26213,Subscriber,NaN,0
3,2411,2016-10-01 00:15:23,2016-10-01 00:55:35,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,-74.032108,26278,Subscriber,1975.0,1
4,170,2016-10-01 00:21:25,2016-10-01 00:24:16,3272,Jersey & 3rd,40.723332,-74.045953,3278,Monmouth and 6th,40.725685,-74.048790,26152,Subscriber,1990.0,1


In [81]:
bike_data["Start Time"] = pd.to_datetime(bike_data["Start Time"])
bike_data["Stop Time"] = pd.to_datetime(bike_data["Stop Time"])
bike_data["hour"] = bike_data["Start Time"].map(lambda x: x.hour)
locations = bike_data.groupby('Start Station ID').first()
locations = locations.loc[:, ["Start Station Latitude",
                              "Start Station Longitude",
                              "Start Station Name"]]
locations.head()


,Start Station Latitude,Start Station Longitude,Start Station Name
Start Station ID,,,
3183,40.716247,-74.033459,Exchange Place
3184,40.714145,-74.033552,Paulus Hook
3185,40.717733,-74.043845,City Hall
3186,40.719586,-74.043117,Grove St PATH
3187,40.721124,-74.038051,Warren St


In [82]:
subset = bike_data[bike_data["hour"]==18]
departure_counts =  subset.groupby("Start Station ID").count()
# select one column
departure_counts = departure_counts.iloc[:,[0]]
# and rename that column
departure_counts.columns= ["Departure Count"]

arrival_counts = subset.groupby("End Station ID").count()
arrival_counts = arrival_counts.iloc[:,[0]]
arrival_counts.columns= ["Arrival Count"]

trip_counts = departure_counts.join(locations).join(arrival_counts)
trip_counts.head()

,Departure Count,Start Station Latitude,Start Station Longitude,Start Station Name,Arrival Count
Start Station ID,,,,,
3183,300,40.716247,-74.033459,Exchange Place,58
3184,80,40.714145,-74.033552,Paulus Hook,33
3185,134,40.717733,-74.043845,City Hall,63
3186,581,40.719586,-74.043117,Grove St PATH,180
3187,90,40.721124,-74.038051,Warren St,83


In [85]:
for index, row in trip_counts.iterrows():
    net_departures = (row["Departure Count"]-row["Arrival Count"])
    radius = net_departures/20
    if net_departures>0:
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    
    
    
    popup_text = """{}<br>
                total departures: {}<br> 
                total arrivals: {}<br>
                net departures: {}"""
    popup_text = popup_text.format(row["Start Station Name"],
                               row["Arrival Count"],
                               row["Departure Count"],
                               net_departures)
    
    folium.CircleMarker(location=(row["Start Station Latitude"],
                                  row["Start Station Longitude"]),
                        radius=radius,
                        color=color,
                        fill=True,
                       popup=popup_text).add_to(folium_map)
    
# folium.CircleMarker(..., popup=popup_text)

folium_map

In [92]:
import numpy as np

def add_lines(image_array, xys, width=1, weights=None):
    """
    Add a set of lines (xys) to an existing image_array
    width: width of lines
    weights: [], optional list of multipliers for lines. 
    """
    
    for i, xy in enumerate(xys):  # loop over lines
        # create a new gray scale image
        image = Image.new("L",(image_array.shape[1], image_array.shape[0]))
        
        # draw the line
        ImageDraw.Draw(image).line(xy, 200, width=width)
        
        #convert to array
        new_image_array = np.asarray(image, dtype=np.uint8).astype(float)
        
        # apply weights if provided
        if weights is not None:
            new_image_array *= weights[i]
            
        # add to existing array
        image_array += new_image_array

    # convolve image
    new_image_array = scipy.ndimage.filters.convolve(image_array, get_kernel(width*4)) 
    return new_image_array

def get_kernel(kernel_size, blur=1/20, halo=.001):
    """
    Create an (n*2+1)x(n*2+1) numpy array.
    Output can be used as the kernel for convolution.
    """
    
    # generate x and y grids
    x, y = np.mgrid[0:kernel_size*2+1, 0:kernel_size*2+1]
    
    center = kernel_size + 1  # center pixel
    r = np.sqrt((x - center)**2 + (y - center)**2)  # distance from center
    
    # now compute the kernel. This function is a bit arbitrary. 
    # adjust this to get the effect you want.
    kernel = np.exp(-r/kernel_size/blur) + (1 - r/r[center,0]).clip(0)*halo
    return kernel

def to_image(array, hue=.62):
    """converts an array of floats to an array of RGB values using a colormap"""
    
    # apply saturation function
    image_data = np.log(array + 1)
    
    # create colormap, change these values to adjust to look of your plot
    saturation_values = [[0, 0], [1, .68], [.78, .87], [0, 1]]
    colors = [hsv_to_rgb([hue, x, y]) for x, y in saturation_values]
    cmap = LinearSegmentedColormap.from_list("my_colormap", colors)
    
    # apply colormap
    out = cmap(image_data/image_data.max())
    
    # convert to 8-bit unsigned integer
    out = (out*255).astype(np.uint8)
    return out

# generate some lines
xys = [(10,10,200,200), (175,10,175,200), (200,10,10,200)]  
weights = np.array([ 2,1,.7])/100  # some weights
# create the image_data
new_image_data = add_lines(np.zeros((220,220)),
                           xys,
                           width=4,
                           weights=weights)
# show the image
Image.fromarray(to_image(new_image_data),mode="RGBA")

NameError: name 'Image' is not defined